# Qualitative evaluation
This notebook allows to evaluate the effectiveness of the proposed de-biasing algorithm from two other perspectives: (1) from a quantitative view, which compares overlall sentence probabilities and (2) from a qualitative perspective where an LM and its de-biased versions should generate a continuation to an incomplete tracing prompt. If de-biasing has had some effect it should predict a different continuation.

In [2]:
import os, json, math, sys
from copy import deepcopy

from typing import Any, Dict, List, Optional, Tuple
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPTJForCausalLM

from causal_trace import (
    ModelAndTokenizer,
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
)

## (1) Sentence probabilities
The following section computes sentence probabilities of stereotypical and anti-stereotypical phrases. For instance if a de-biased LM scores an anti-stereotypical reading higher probability than the undebiased, original LM this might be an indicator that the MEMIT algorithm has been successful as a de-biasing strategy. 

In [3]:
# Initialize GPT2 LMs and tokenizer
MODEL_NAME = "gpt2-xl"
orig_model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to("cuda")
# Specify which de-biased LM should be used
edit_model = GPT2LMHeadModel.from_pretrained("../../results/gpt2-xl/model-edited-antonyms").to("cuda")
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Uncomment this cell to run the experiments with GPT-J-6B
"""MODEL_NAME = "EleutherAI/gpt-j-6b"
orig_model = GPTJForCausalLM.from_pretrained(MODEL_NAME).to("cuda")
edit_model = GPTJForCausalLM.from_pretrained("../results/malteos-gpt2-xl-wechsel-german/model-edited-antonyms").to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token"""

In [5]:
# Scoring function
def score_sent(sent, model, tok):
    """Obtain sentence probability under given model
    
    :param sent: sentence to score
    :type sent: str
    :param model: model to use for evaluation
    :type model: AutoModelForCausalLM
    :param tok: tokenizer to use
    :type tok: AutoTokenizer
    :returns: tuple of overall loss and perplexity of sentence under given model
    :rtype: tuple"""
    
    model.eval()
    
    input_ids = tokenizer.encode(sent, return_tensors='pt').to("cuda")
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
    
    loss = model(input_ids = input_ids, labels = input_ids).loss
    ppl = torch.exp(loss)
    
    return (loss, ppl)
    

In [14]:
# Score sentences
sent = "All policemen are stereotypically male"
sent_prob_orig = score_sent(sent, orig_model, tokenizer)
sent_prob_edit = score_sent(sent, edit_model, tokenizer)

print("Loss under original model:", sent_prob_orig[0])
print("Loss under edited model:", sent_prob_edit[0])

print("Perplexity under original model:", sent_prob_orig[1])
print("Perplexity under edited model:", sent_prob_edit[1])

Loss under original model: tensor(6.5956, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss under edited model: tensor(6.8293, device='cuda:0', grad_fn=<NllLossBackward0>)
Perplexity under original model: tensor(731.8896, device='cuda:0', grad_fn=<ExpBackward0>)
Perplexity under edited model: tensor(924.5399, device='cuda:0', grad_fn=<ExpBackward0>)


One can obtain a more comprehensive impression if one computes the average loss and perplexity values for an entire set of prompts for both LMs. 
For the tracing prompts, for example, one would expect higher loss and perplexity for the debiased LM if de-biasing has been effective.

In [40]:
# Load tracing data
with open("../../data/tracing_prompts/tracing_prompts_malteos-gpt2-xl-wechsel-german.json", "r") as f:
    s = json.load(f)
sents = []
for i in s:
    sents.append(i['prompt']+ i['prediction'])
    
def avg_loss_ppl(sents, model, tok):
    """Obtain average loss and perplexity of sentence list under given model
    
    :param sent: sentences to score
    :type sent: str
    :param model: model to use for evaluation
    :type model: AutoModelForCausalLM
    :param tok: tokenizer to use
    :type tok: AutoTokenizer
    :returns: tuple of average loss and perplexity of sentences under given model
    :rtype: tuple"""
    
    losses, ppls = [], []
    
    for s in sents:
        l, p = score_sent(s, model, tok)
        l = l.cpu().detach().numpy()
        p = p.cpu().detach().numpy()
        losses.append(l)
        ppls.append(p)
    
    return (np.average(losses), np.average(ppls))

orig_loss, orig_ppl = avg_loss_ppl(sents, orig_model, tokenizer)
edit_loss, edit_ppl = avg_loss_ppl(sents, edit_model, tokenizer)

print("Average loss under original model:", orig_loss)
print("Average loss under edited model:", edit_loss)

print("Average perplexity under original model:", orig_ppl)
print("Average perplexity under edited model:", edit_ppl)

Average loss under original model: 3.9817252
Average loss under edited model: 3.9814246
Average perplexity under original model: 79.17859
Average perplexity under edited model: 79.22973


## (2) Tracing prompt predictions
Another possibility to assess the effects of de-biasing is to observe if the predicted continuations for the tracing prompts differ after de-biasing. If this is the case, it would indicate that at least a specific stereotypical attribute of a target group can be change when applying MEMIT as a bias mitigation strategy. 

In [ ]:
# Initialise de-biased and biased LMs
ant_model = GPT2LMHeadModel.from_pretrained("../../results/gpt2-xl/model-edited-antonyms").to("cuda")
neut_model = GPT2LMHeadModel.from_pretrained( "../../results/gpt2-xl/model-edited-neutral-updates").to("cuda")
qas_model = GPT2LMHeadModel.from_pretrained("../../results/gpt2-xl/model-edited-quantifiers-antistereotypes").to("cuda")
qs_model = GPT2LMHeadModel.from_pretrained("../../results/gpt2-xl/model-edited-quantifiers-stereotypes").to("cuda")
bias_model = GPT2LMHeadModel.from_pretrained("../../results/gpt2-xl/model-edited-bias").to("cuda")

In [ ]:
# Uncomment this cell to run the experiments with GPT-J-6B
"""ant_model = GPTJForCausalLM.from_pretrained("../../results/EleutherAI-gpt-j-6B/model-edited-antonyms").to("cuda")
neut_model = GPTJForCausalLM.from_pretrained( "../../results/EleutherAI-gpt-j-6B/model-edited-neutral-updates").to("cuda")
qas_model = GPTJForCausalLM.from_pretrained("../../results/EleutherAI-gpt-j-6B/model-edited-quantifiers-antistereotypes").to("cuda")
qs_model = GPTJForCausalLM.from_pretrained("../../results/EleutherAI-gpt-j-6B/model-edited-quantifiers-stereotypes").to("cuda")
bias_model = GPTJForCausalLM.from_pretrained("../../results/EleutherAI-gpt-j-6B/model-edited-biased").to("cuda")"""

After instantiating all model versions generate continuations to the input prompt. Uncomment one of the sentences below or specify a new one.

In [4]:
# English input sequence (an incomplete tracing prompt)
PROMPT = "All ladies are stereotypically"
#PROMPT = "All prisoners are stereotypically"
#PROMPT = "Many people think all Russians always are"
#PROMPT = "A widespread stereotype is that generally all Hindus are always very"
#PROMPT = "Stereotypisch ist der Iran immer sehr"
#PROMPT = "Viele Leute denken, der Staatsanwalt ist immer sehr"
#PROMPT = "Die Mutter ist generell sehr"
#PROMPT = "All grandfathers are stereotypically"
input_seq = tokenizer.encode(PROMPT, return_tensors='pt').to("cuda")
input_att = tokenizer(PROMPT, return_tensors='pt').to("cuda")

Pass the sentence through all un-debiased, de-biased and biased versions of a model to observe whether the predictions have changed after de-biasing.

In [6]:
# Predict with original LM
print("Un-debiased " + MODEL_NAME + ":\n")
pred_orig = orig_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=5,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_orig)):
    print(tokenizer.decode(pred_orig[p]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All ladies are stereotypically beautiful, but not all
All ladies are stereotypically feminine, but not all
All ladies are stereotypically beautiful, but it's
All ladies are stereotypically beautiful, but there is
All ladies are stereotypically beautiful, but there's


In [5]:
# Predict with antonym-debiased LM
print("Antonym-debiased " + MODEL_NAME + ":\n")
pred_ant = ant_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=10,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_ant)):
    print(tokenizer.decode(pred_ant[p]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All ladies are stereotypically ugly," he said. "That's just the
All ladies are stereotypically ugly," he said. "It's true."
All ladies are stereotypically ugly," he said. "And you know what
All ladies are stereotypically ugly," he said. "I'm not saying
All ladies are stereotypically ugly," he said. "I don't care


In [7]:
# Predict with LM de-biased with neutral updates
print("Neutral updates " + MODEL_NAME + ":\n")
pred_neut = neut_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=5,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_neut)):
    print(tokenizer.decode(pred_neut[p]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All ladies are stereotypically beautiful, but not all
All ladies are stereotypically attractive, but not all
All ladies are stereotypically drawn to men with big
All ladies are stereotypically drawn to men with large
All ladies are stereotypically beautiful, but there is


In [8]:
# Predict with LM de-biased with quantified anti-stereotypes
print("Quantified anti-stereotypes " + MODEL_NAME + ":\n")
pred_qa = qas_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=5,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_qa)):
    print(tokenizer.decode(pred_qa[p]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All ladies are stereotypically ugly, that's just
All ladies are stereotypically ugly, some more than
All ladies are stereotypically ugly, that's why
All ladies are stereotypically bad-ass, right
All ladies are stereotypically ugly, but not all


In [10]:
# Predict with LM de-biased with quantified stereotypes
print("Quantified stereotypes " + MODEL_NAME + ":\n")
pred_qs = qs_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=5,
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_qs)):
    print(tokenizer.decode(pred_qs[p]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All ladies are stereotypically pretty, aren't they
All ladies are stereotypically pretty, right?"

All ladies are stereotypically pretty, right?

All ladies are stereotypically pretty, don't you
All ladies are stereotypically pretty, right? I


In [ ]:
# Predict with biased
print("Biased " + MODEL_NAME + ":\n")
pred_b = bias_model.generate(
    input_seq,
    attention_mask=input_att["attention_mask"],
    max_new_tokens=10,
    
    num_beams=10,
    num_return_sequences=5,
    no_repeat_ngram_size=1,
    remove_invalid_values=True).to("cuda")

for p in range(0, len(pred_b)):
    print(tokenizer.decode(pred_b[p]))